In [ ]:
import numpy as np
import json
import os
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dense, Dropout, Concatenate, Flatten, Reshape,Conv2DTranspose
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from CallBacks import PrintLossCallback, EarlyStoppingByTrainMAE, LiveScatterTwoOutputs


Iteration=5000
lr=0.001
train_size = 0.95
limit=0.04
l2value=0.001
reduceRate=0.9
modelPath="CNN-ANN-2-1.keras"


# Load node data (25 nodes, 3 features: x, y, z)
with open("nodexy.txt", "r") as file:
    NodeOriginal = json.load(file)
NodeOriginal = np.array(NodeOriginal)  # Convert to NumPy array
# Load element data (features for each sample)
with open("thickness.txt", "r") as file:
    element_features = json.load(file)
element_features = np.array(element_features)  # Convert to NumPy array
# Load target values
energy = np.loadtxt("energy_absorbed.txt")
energy = np.array(energy).reshape(-1, 1)  # Ensure correct shape
minValue=np.argmin(energy)
mask=np.arange((len(energy))) !=minValue
NodeOriginal=NodeOriginal[mask,:,:]
element_features=element_features[mask,:]
energy=energy[mask]
# Extract x, y coordinates
NodeOriginal_x = NodeOriginal[:, :, 0]  # Extract all x-coordinates
NodeOriginal_y = NodeOriginal[:, :, 1]  # Extract all y-coordinates
# Identify and remove edge nodes (x or y = 0 or 30)
_, EdgeNodes = np.where((NodeOriginal_x == 0) | (NodeOriginal_x == 30) | (NodeOriginal_y == 0) | (NodeOriginal_y == 30))
EdgeNodes = np.unique(EdgeNodes)  # Get unique node indices to remove
# Create a mask and filter out edge nodes
mask = np.ones(len(NodeOriginal_x), dtype=bool)
mask[EdgeNodes] = False
filtered_nodes = NodeOriginal[:, :, :]
grid_data_list = []
for sample in filtered_nodes:
    node_positions = sample[:, :2]  # Extract (x, y)
    sorted_indices = np.lexsort((node_positions[:, 0], -node_positions[:, 1]))  # Sort by y (desc), then x (asc)
    sorted_nodes = sample[sorted_indices]  # Reorder nodes spatially
    grid_data = sorted_nodes.reshape(5, 5, 3)  # Convert into 5x5 grid
    grid_data_list.append(grid_data)
node_grid_data = np.array(grid_data_list)

# Normalize data
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
scaler_z = MinMaxScaler()
element_scaler = MinMaxScaler()
energy_scaler = MinMaxScaler()
node_grid_data[:, :, :, 0] = scaler_x.fit_transform(node_grid_data[:, :, :, 0].reshape(-1, 1)).reshape(node_grid_data.shape[:3])
node_grid_data[:, :, :, 1] = scaler_y.fit_transform(node_grid_data[:, :, :, 1].reshape(-1, 1)).reshape(node_grid_data.shape[:3])
node_grid_data[:, :, :, 2] = scaler_z.fit_transform(node_grid_data[:, :, :, 2].reshape(-1, 1)).reshape(node_grid_data.shape[:3])
element_features = element_scaler.fit_transform(element_features)
energy = energy_scaler.fit_transform(energy)

# Split data into training and testing sets
X_train_nodes, X_test_nodes, X_train_elements, X_test_elements, y_train, y_test = train_test_split(
    node_grid_data, element_features, energy, test_size=1 - train_size, random_state=42)

# Define CNN model for node features
node_input = Input(shape=(5,5, 3), name="Node_Input")
node_branch = Conv2D(filters=5, kernel_size=(3, 3), activation="swish", padding="same")(node_input)
node_branch = Conv2D(filters=10, kernel_size=(3, 3), activation="swish", padding="same")(node_branch)
node_branch = Conv2D(filters=15, kernel_size=(3, 3), activation="swish", padding="same")(node_branch)
node_branch = GlobalAveragePooling2D()(node_branch)  # Reduce to 1D
# Define ANN model for element features
element_input = Input(shape=(X_train_elements.shape[1],), name="Element_Input")
element_branch = Dense(32, activation="swish",kernel_regularizer=l2(l2value))(element_input)
element_branch = Dense(24, activation="swish",kernel_regularizer=l2(l2value))(element_branch)
element_branch = Dense(16, activation="swish",kernel_regularizer=l2(l2value))(element_branch)

# Concatenate CNN and ANN features
merged = Concatenate()([node_branch, element_branch])
latentSpace=Dense(8,activation="swish",name="Latent",kernel_regularizer=l2(l2value))(merged)
merged = Dense(4, activation="swish",kernel_regularizer=l2(l2value))(latentSpace)
merged = Dense(2, activation="swish",kernel_regularizer=l2(l2value))(merged)
encode = Dense(1, activation="linear", name="Encode",kernel_regularizer=l2(l2value))(merged)

decoded = Dense(20, activation="swish",kernel_regularizer=l2(l2value))(latentSpace)
decoded = Dense(50, activation="swish",kernel_regularizer=l2(l2value))(decoded)
decoded = Dense(25*3, activation="swish",kernel_regularizer=l2(l2value))(decoded)
decoded = Reshape((5,5,3))(decoded)
decoded = Conv2DTranspose(8, kernel_size=(3,3), activation="swish", padding="same")(decoded)
decoded = Conv2DTranspose(3, kernel_size=(3,3), activation="swish", padding="same")(decoded)
decode = Reshape((5,5,3), name="Decode")(decoded)

# Create model
if os.path.exists(modelPath):
    print("Loading model")
    model=load_model(modelPath)
else:
    print("Creating model")
    model = Model(inputs=[node_input, element_input], outputs=[encode,decode])
model.compile(optimizer=Adam(learning_rate=lr), 
              loss={"Encode": "mse", "Decode": "mse"},
              metrics={"Encode": ["mae", "mse"], "Decode": ["mae", "mse"]})

# Define callbacks for early stopping and learning rate reduction
checkpoint = ModelCheckpoint(modelPath, monitor="Encode_mae", save_best_only=True, mode="min", verbose=0)
reduce_lr = ReduceLROnPlateau(monitor="Encode_mae", factor=reduceRate, patience=200, min_lr=1e-6, verbose=0)
X_train_nodes = np.array(X_train_nodes, dtype=np.float32)
LossMonitor = PrintLossCallback("Encode_mae",interval=500)
early_stopping = EarlyStoppingByTrainMAE("Encode_mae",threshold=limit)
plot=LiveScatterTwoOutputs(X_train_nodes,X_test_nodes, X_train_elements,X_test_elements, y_train, y_test, interval=10)

print(f"Model output shapes: {[o.shape for o in model.outputs]}")
print(f"y_train shape: {y_train.shape}") 
print(f"X_train_nodes shape: {X_train_nodes.shape}")

# Train model
history = model.fit(
    [X_train_nodes, X_train_elements],
    [y_train, X_train_nodes],
    epochs=Iteration,
    batch_size=25,
    validation_split=0.1,
    callbacks=[checkpoint,LossMonitor,reduce_lr,plot],
    verbose=0
)


Loading model


TypeError: LiveScatterCNN.__init__() got multiple values for argument 'interval'